In [1]:
import pandas as pd
from sklearn import datasets # sklearn comes with some toy datasets to practise
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import sys
import numpy as np
# Make sure that your config.py Python file with your credentials is located
# in the same folder as your Jupyter notebook.
from config import *
import pickle

In [2]:
from sklearn.datasets import make_classification
from sklearn.cluster import HDBSCAN
from scipy.spatial.distance import euclidean
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from kneed import KneeLocator
import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc
import time

In [3]:
from functions import*

In [4]:
from IPython.display import clear_output

In [5]:
from scipy.spatial import distance_matrix

In [6]:
from IPython.display import display, HTML

In [7]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentias #
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=Client_ID,
                                                           client_secret=Client_Secret))

In [8]:
sp._session.timeout = 10

In [16]:
import pickle

def main():
    
    # We import the scalers and required df
    
    scaler_filename = "./scaler_v1.pickle"
    with open(scaler_filename, "rb") as file:
        scaler = pickle.load(file)
        
    umap_filename = "./umap.pickle"
    with open(umap_filename, "rb") as file:
        umap = pickle.load(file)
        
    database_filename = "./umap_df.csv"
    X_umap_df = pd.read_csv(database_filename)
    
    all_songs_filename = "../../Day_3/Afternoon/lab-clustering-songs/all_songs_clustered.csv"
    all_songs = pd.read_csv(all_songs_filename)
    all_songs['open_link'] = all_songs['id'].apply(lambda track_id: f'<a href="https://open.spotify.com/track/{track_id}" target="_blank">Click me</a>')
      
    answer = True
    while answer == True:
        # Ask the user to enter a song
        
        user_input = input("Please enter your song: ")
        
        # Based on the users input we show a list of 5 song matches among which the user has to select
        
        five_songs = search_song_five(user_input)
        if five_songs.empty:
            print("Sorry, no songs found. Please enter a different song.")
            continue  # Ask for a new input if no songs are found
        five_songs_no_id=five_songs.drop(columns="id")
        display(five_songs_no_id)
        
        # The user returns an int among 0 and 4, which is then used to retrieve the song id from the five_songs df
        
        select_track = int(input("Please select your track from, giving a number between 0 and 4:"))
        song_id = five_songs.iloc[select_track,2]
        
        # With the song id we extract the audio features for the song
        
        audio_feat=get_audio_features_for_chunks(sp, song_id,sleep_time=0)
        
        # To proceed with the modelling, we extract the numerical values of the audio_feat df:
        
        X = audio_feat.select_dtypes(np.number)
        
        # We apply the previous loaded umap dimensionality reduction, which seems to be the best performing one and convert it into a DataFrame:
        
        umap_representation = umap.transform(X)
                
        user_song_umap_df= pd.DataFrame(umap_representation,columns=["UMAP_1","UMAP_2"])
        
        # Since the DBSCAN does not have fit method, we calculate the distance among the user´s choice and all 
        #the songs with the method distance_matrix from scipy.spatial.distance:
        
        d = distance_matrix(user_song_umap_df, X_umap_df.iloc[:,[0,1]]) #numpy array
        closest_song_to_user_song = np.argmin(d)
        
        # We retrieve the cluster from the all_songs df, using the index obtained for the closest record measured.
        
        cluster=X_umap_df.iloc[closest_song_to_user_song,-1]
        
        # Passing the function get_records_by_id_and_cluster we check if the song id is within the "hot" flagged songs. 
        # If the song is "hot" we return 5 song suggestions flagged as "hot" from the same cluster
        # If the song is "not_hot" we return 5 song suggestions flagged as "not_hot" from the same cluster
        
        recommended_songs=get_records_by_id_and_cluster(all_songs,song_id,cluster)
        recommended_songs = recommended_songs[["artist","title","open_link"]]
        
        print("Please check the recommendations we have selected for you:\n\n")
        
        # We use the method .to_html to make the url link of the song clickable:
        
        display(HTML(recommended_songs.to_html(escape=False)))

        # Ask the user if they want more recommendations
        user_response = input("Do you want to get more songs recommended? (yes/no): ").lower()
        while user_response not in ["yes", "no"]:
            print("\n")
            print("Please enter (yes/no)\n")
            user_response = input("Do you want to get more songs recommended? (yes/no): ").lower()
            print("\n")

        # Check if the user wants more recommendations
        if user_response == 'no':
            answer = False

In [22]:
main()

Please enter your song: pretender


,title,artist
0,The Pretender,Foo Fighters
1,The Pretender,Lewis Capaldi
2,The Pretender,Lewis Capaldi
3,The Pretender,Jackson Browne
4,The Pretender,Lewis Capaldi


Please select your track from, giving a number between 0 and 4:0
Please check the recommendations we have selected for you:




,artist,title,open_link
167,esther phillips,just say goodbye,Click me
1621,astrud gilberto,didn't we,Click me
1553,dean martin,blue smoke (kohu-auwahi),Click me
2794,trapt,bring it,Click me
1218,del shannon,cry myself to sleep,Click me


Do you want to get more songs recommended? (yes/no): yes
Please enter your song: cadillac solitario


,title,artist
0,Cadillac solitario,Loquillo
1,Cadillac solitario,Loquillo
2,Cadillac solitario - Live,"Loquillo Y Los Trogloditas, Loquillo"
3,Cadillac Solitario - Bonus Track,Buitres
4,Cadillac Solitario,Clara Roldan


Please select your track from, giving a number between 0 and 4:1
Please check the recommendations we have selected for you:




,artist,title,open_link
2927,sister rosetta tharpe,rock me,Click me
1415,shirley bassey,till,Click me
876,josé gonzález,heartbeats,Click me
1013,johnny cash,after the ball,Click me
1032,"grover washington, jr.",be mine (tonight) (feat. grady tate),Click me


Do you want to get more songs recommended? (yes/no): ss


Please enter (yes/no)

Do you want to get more songs recommended? (yes/no): tt




Please enter (yes/no)

Do you want to get more songs recommended? (yes/no): no


